In [2]:
import numpy as np
import pandas as pd
import math
from arbiter_puf import ArbiterPuf
from xor_puf import XorPuf
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score


In [3]:
#  drops responses in Y  adds responses in X
def modify_data_to_train(data):
    training_data = []
    test_data = []
    for i in data:
        test_data.append(i.pop())
        training_data.append(i)
    return np.asarray(training_data), np.asarray(test_data)

In [23]:
list_of_number_of_bits = [32, 64]
data_size = 100000
print('================== Arbiter PUF ========================')
for bit in list_of_number_of_bits:
    print('Number of Stages: ', bit)
    dataset = ArbiterPuf(bit).calculate_responses(data_size)
    X,Y = modify_data_to_train(dataset)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    LGR_Classifier = LogisticRegression()
    LGR_Classifier.fit(X_train, Y_train)
    scores = cross_val_score(LGR_Classifier, X_train, Y_train, cv=10, scoring="accuracy")
    y_pred = LGR_Classifier.predict(X_test)
    print ("Cross Validation Mean Score: ", '{}%'.format(np.round(scores.mean(), 3) * 100))  
    print("Accuracy : ", accuracy_score(Y_test, y_pred))
    print(classification_report(Y_test, y_pred))
    print('=======================================================')


================== Arbiter PUF ========================
Number of Stages:  32
Cross Validation Mean Score:  50.1%
Accuracy :  0.50215
              precision    recall  f1-score   support

           0       0.50      0.48      0.49     10048
           1       0.50      0.52      0.51      9952

    accuracy                           0.50     20000
   macro avg       0.50      0.50      0.50     20000
weighted avg       0.50      0.50      0.50     20000

Number of Stages:  64
Cross Validation Mean Score:  56.39999999999999%
Accuracy :  0.56905
              precision    recall  f1-score   support

           0       0.58      0.57      0.57     10113
           1       0.56      0.57      0.57      9887

    accuracy                           0.57     20000
   macro avg       0.57      0.57      0.57     20000
weighted avg       0.57      0.57      0.57     20000



In [4]:
def modify_data(data): # adding phi to the dataset when training
    training_data = []
    test_data = []
    for i in data:
        test_data.append(i.pop())
        phi = [ArbiterPuf.calculate_phi(i[j:]) for j in range(len(i))]
        training_data.append(phi)   
    return np.asarray(training_data), np.asarray(test_data)


In [5]:
list_of_number_of_bits = [32, 64]
data_size = 100000
print('================== Arbiter PUF with Phi ========================')
for bit in list_of_number_of_bits:
    print('Number of Stages: ', bit)
    dataset = ArbiterPuf(bit).calculate_responses(data_size)
    X,Y = modify_data(dataset)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    LGR_Classifier = LogisticRegression()
    LGR_Classifier.fit(X_train, Y_train)
    scores = cross_val_score(LGR_Classifier, X_train, Y_train, cv=10, scoring="accuracy")
    y_pred = LGR_Classifier.predict(X_test)
    print ("Cross Validation Mean Score: ", '{}%'.format(np.round(scores.mean(), 3) * 100))  
    print("Accuracy : ", accuracy_score(Y_test, y_pred))
    print(classification_report(Y_test, y_pred))
    print('=======================================================')


================== Arbiter PUF with Phi ========================
Number of Stages:  32
Cross Validation Mean Score:  99.9%
Accuracy :  0.9985
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10037
           1       1.00      1.00      1.00      9963

    accuracy                           1.00     20000
   macro avg       1.00      1.00      1.00     20000
weighted avg       1.00      1.00      1.00     20000

Number of Stages:  64
Cross Validation Mean Score:  99.9%
Accuracy :  0.99775
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10061
           1       1.00      1.00      1.00      9939

    accuracy                           1.00     20000
   macro avg       1.00      1.00      1.00     20000
weighted avg       1.00      1.00      1.00     20000



In [ ]:
list_of_number_of_bits = [5, 10, 15, 20, 32]
data_size = 100000
number_of_pufs = [3, 4, 5]
for j in number_of_pufs:
    print(f"================== Xor PUF {j} Pufs ========================")
    for bit in list_of_number_of_bits:
        print('Number of Stages: ', bit)
        # dataset = ArbiterPuf(bit).calculate_responses(data_size)
        dataset = XorPuf(bit, j).calculate_responses(data_size)
        X,Y = modify_data(dataset)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
        LGR_Classifier = LogisticRegression()
        LGR_Classifier.fit(X_train, Y_train)
        scores = cross_val_score(LGR_Classifier, X_train, Y_train, cv=10, scoring="accuracy")
        y_pred = LGR_Classifier.predict(X_test)
        print ("Cross Validation Mean Score: ", '{}%'.format(np.round(scores.mean(), 3) * 100))  
        print("Accuracy : ", accuracy_score(Y_test, y_pred))
        print(classification_report(Y_test, y_pred))
        print('=======================================================')


================== Xor PUF 3 Pufs ========================
Number of Stages:  5
Cross Validation Mean Score:  85.2%
Accuracy :  0.87265
              precision    recall  f1-score   support

           0       0.87      0.87      0.87      9999
           1       0.87      0.87      0.87     10001

    accuracy                           0.87     20000
   macro avg       0.87      0.87      0.87     20000
weighted avg       0.87      0.87      0.87     20000

Number of Stages:  10
Cross Validation Mean Score:  63.0%
Accuracy :  0.6303
              precision    recall  f1-score   support

           0       0.63      0.65      0.64      9978
           1       0.64      0.61      0.62     10022

    accuracy                           0.63     20000
   macro avg       0.63      0.63      0.63     20000
weighted avg       0.63      0.63      0.63     20000

Number of Stages:  15
